# Preparation

In [1]:
import time
prep_start_time = time.time()

In [1]:
!mkdir pretrained
!mkdir dataset
!mkdir logs

mkdir: pretrained: File exists
mkdir: dataset: File exists
mkdir: logs: File exists


In [3]:
!rm -r pretrained/mobilenet
!mkdir pretrained/mobilenet
!wget https://storage.googleapis.com/mobilenet_v2/checkpoints/mobilenet_v2_1.0_224.tgz -O pretrained/mobilenet_v2_1.0_224.tgz
!tar -xvf pretrained/mobilenet_v2_1.0_224.tgz -C pretrained/mobilenet/
!rm pretrained/mobilenet_v2_1.0_224.tgz

rm: cannot remove 'pretrained/mobilenet': No such file or directory
--2018-08-29 01:31:47--  https://storage.googleapis.com/mobilenet_v2/checkpoints/mobilenet_v2_1.0_224.tgz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.26.48, 2404:6800:4004:801::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.26.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78306834 (75M) [application/x-tar]
Saving to: ‘pretrained/mobilenet_v2_1.0_224.tgz’

pretrained/mobilene 100%[===================>]  74.68M  36.3MB/s    in 2.1s    

2018-08-29 01:31:50 (36.3 MB/s) - ‘pretrained/mobilenet_v2_1.0_224.tgz’ saved [78306834/78306834]

./
./mobilenet_v2_1.0_224_eval.pbtxt
./mobilenet_v2_1.0_224.ckpt.data-00000-of-00001
./mobilenet_v2_1.0_224_frozen.pb
./mobilenet_v2_1.0_224.tflite
./mobilenet_v2_1.0_224_info.txt
./mobilenet_v2_1.0_224.ckpt.meta
./mobilenet_v2_1.0_224.ckpt.index


In [1]:
!rm -r pretrained/resnet
!mkdir pretrained/resnet
!wget http://download.tensorflow.org/models/resnet_v2_101_2017_04_14.tar.gz -O pretrained/resnet_v2_101_2017_04_14.tar.gz
!tar -xvf pretrained/resnet_v2_101_2017_04_14.tar.gz -C pretrained/resnet/
!rm pretrained/resnet_v2_101_2017_04_14.tar.gz

rm: pretrained/resnet: No such file or directory
--2018-07-10 15:10:59--  http://download.tensorflow.org/models/resnet_v2_101_2017_04_14.tar.gz
Resolving download.tensorflow.org... 172.217.161.48
Connecting to download.tensorflow.org|172.217.161.48|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 499948384 (477M) [application/x-tar]
Saving to: ‘pretrained/resnet_v2_101_2017_04_14.tar.gz’

pretrained/resnet_v 100%[===================>] 476.79M  10.8MB/s    in 49s     

2018-07-10 15:11:49 (9.78 MB/s) - ‘pretrained/resnet_v2_101_2017_04_14.tar.gz’ saved [499948384/499948384]

x resnet_v2_101.ckpt
x train.graph
x eval.graph


In [ ]:
!mkdir dataset/coco
!wget http://images.cocodataset.org/zips/train2017.zip -O dataset/coco/train2017.zip
!wget http://images.cocodataset.org/zips/val2017.zip -O dataset/coco/val2017.zip
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip -O dataset/coco/annotations_trainval2017.zip
!unzip dataset/coco/*.zip

In [3]:
!pip install scikit-image
!pip install pycocotools

You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
